In [1]:
from utils.load_models import *
from utils.novelty_serve import *

In [2]:
model, lang, model_conf = load_han_novelty("NOV-264")
model_conf.num_sent = 100
model_dict = {"model": model.cuda(), "lang": lang, "model_conf": model_conf}

In [3]:
import pandas as pd

data = pd.read_csv("dlnd_data.csv")

In [4]:
row = data.iloc[1]
target_text = row.target_text
source_text = row.source
true_class = row.DLA

In [5]:
opt, src_att, trg_att = map_attention(source_text, target_text, model_dict)

100%|██████████| 12/12 [00:00<00:00, 930.02it/s]


In [11]:
import pandas as pd
import numpy as np
import html
import random
from IPython.core.display import display, HTML
from IPython.display import IFrame
import numpy as np
import matplotlib
import matplotlib.pyplot as plt


def colorize(words, color_array):
    cmap=matplotlib.cm.Reds
    template = '<span class="barcode"; style="color: black; background-color: {}">{}</span>'
    colored_string = ''
    for word, color in zip(words, color_array):
        color = matplotlib.colors.rgb2hex(cmap(color)[:3])
        colored_string += template.format(color, '&nbsp' + word + '&nbsp')
    return colored_string




In [12]:
def plot(sent,attn):
    source = nltk.sent_tokenize(sent)
    if torch.is_tensor(attn):
        attn = attn.detach().cpu().numpy().flatten()
    attn_weights = 3*attn[0:len(source)]
    s = colorize(source,attn_weights)
    
    # or simply save in an html file and open in browser
    with open('colorize.html', 'w') as f:
        f.write(s)


In [106]:
# 1502 - too much attention on one sentence
# 1690 - To much information in source (mostly repetetive)
# 1778 - Attention too localized
# 3382 - Too much information in source
#[1502, 1690, 1778, 3382, 3761, 881, 4514, 2689]

row = data.iloc[3382]
target_text = row.target_text
source_text = row.source
true_class = row.DLA
print("True")
print(true_class)

True
Non-Novel


In [107]:
opt, src_att, trg_att = map_attention(source_text, target_text, model_dict)

100%|██████████| 10/10 [00:00<00:00, 1284.90it/s]


In [108]:
print("Pred",opt[0])
print(("Novel" if opt[0][1]>0.5 else 'Non-Novel'))

Pred [0.3736461102962494, 0.626353919506073]
Novel


In [109]:
plot(source_text,src_att)
IFrame('./colorize.html',width=1200,height=400)



In [110]:
plot(target_text,trg_att)
IFrame('./colorize.html',width=1200,height=400)


# Errros by DAN correctly predicted by our model (HAN-CNN)

In [18]:
import pandas as pd

dan_analysis = pd.read_csv('./analysis/dan_analysis_report.csv')
han_cnn_analysis = pd.read_csv('./analysis/han_cnn_analysis_report.csv')

In [19]:
import nltk

def sent_analysis(df):
    # target analysis
    print("Num sentences")
    print()
    target = df.target_text.apply(lambda x: nltk.sent_tokenize(x))
    target_len = target.apply(lambda x: len(x))
    print("Max num sentences in target -",max(target_len))
    print("Avg num sentences in target -",target_len.mean())
    print()
    source = df.source.apply(lambda x: nltk.sent_tokenize(x))
    source_len = source.apply(lambda x: len(x))
    print("Max num sentences in source -",max(source_len))
    print("Avg num sentences in source -",source_len.mean())
    

sent_analysis(dan_analysis[dan_analysis.id.isin(set(dan_analysis.id) - (set(dan_analysis.id) - set(han_cnn_analysis.id)))])

Num sentences

Max num sentences in target - 45
Avg num sentences in target - 13.525

Max num sentences in source - 91
Avg num sentences in source - 47.7


In [20]:
print("Errors by DAN - correctly predicted by HAN-CNN")
sent_analysis(dan_analysis[dan_analysis.id.isin(set(dan_analysis.id) - (set(dan_analysis.id) - set(han_cnn_analysis.id)))])

Errors by DAN - correctly predicted by HAN-CNN
Num sentences

Max num sentences in target - 45
Avg num sentences in target - 13.525

Max num sentences in source - 91
Avg num sentences in source - 47.7


In [21]:
print("Errors by DAN")
sent_analysis(dan_analysis)

Errors by DAN
Num sentences

Max num sentences in target - 45
Avg num sentences in target - 13.776699029126213

Max num sentences in source - 152
Avg num sentences in source - 47.99029126213592


In [22]:
print("Errors by DAN")
sent_analysis(han_cnn_analysis)

Errors by DAN
Num sentences

Max num sentences in target - 45
Avg num sentences in target - 13.606060606060606

Max num sentences in source - 111
Avg num sentences in source - 49.22727272727273


In [24]:
print(dan_analysis[dan_analysis.id.isin(set(dan_analysis.id) - (set(dan_analysis.id) - set(han_cnn_analysis.id)))].id.values)

[  12  271  352  550  724  832  916 1176 1196 1198 1587 1732 1740 1961
 2072 2158 2172 2305 2669 2822 2916 3281 3312 3642 3666 3730 3928 3945
 3950 4045 4077 4082 4187 4361 4398 4428 4915 4924 5210 5216]


In [69]:
x="""In the middle of a crowded bar, Adam Purinton yelled at two Indian men to “ get out of my country ,” witnesses said, then opened fire in an attack that killed one of the men and wounded the other, as well as a third man who tried to help. 
"""
data[data['target_text']==x]

,Unnamed: 0,target_text,source,DLA,id
1305,1305,"In the middle of a crowded bar, Adam Purinton ...",Law enforcement authorities in Johnson County...,Non-Novel,1305


In [75]:
ids=[]
for i in dan_analysis[dan_analysis.id.isin(set(dan_analysis.id) - (set(dan_analysis.id) - set(han_cnn_analysis.id)))].target_text.values:
    ids.append(data[data['target_text']==i].id.values[0])
print(ids)

[1502, 1319, 1628, 1124, 1980, 1690, 2124, 1831, 1778, 1851, 3999, 3382, 3371, 4066, 3133, 3780, 3761, 3683, 815, 567, 881, 4454, 4514, 4588, 2227, 2224, 50, 44, 79, 137, 5171, 5175, 5252, 5223, 4887, 5053, 2689, 2687, 2347, 2487]


In [94]:

my_ids =[]

for id in ids:    
    row = data.iloc[id]
    target_text = row.target_text
    source_text = row.source
    true_class = row.DLA
    opt, src_att, trg_att = map_attention(source_text, target_text, model_dict)
    pred = ("Novel" if opt[0][1]>0.5 else 'Non-Novel')
    if pred!=true_class:
        my_ids.append(id)
    

100%|██████████| 43/43 [00:00<00:00, 1374.02it/s]


In [95]:
my_ids

[1502, 1690, 1778, 3382, 3761, 881, 4514, 2689]